#### Download the file with the summary of the accommodations: [listings.csv](https://raw.githubusercontent.com/tnavarrete-iedib/bigdata-24-25/refs/heads/main/listings.csv). We are going to focus on the price per night (_price_ column).

#### 3.1. Show the boxplot of the price per night.

#### 3.2. Calculate the average price and its median. Detect the outliers and replace their values ​​with the average price you have calculated. We will consider as an outlier the one that is separated from the average by more than 2 times the standard deviation. Then, calculate the average and median again. Comment on whether they have changed and, if so, how.

#### 3.3. Calculate the average price (once the outliers have been replaced) per night for each of the municipalities on the island and order them from most expensive to cheapest.

#### Now we will also work with the detailed comments file: [reviews.csv.gz](https://github.com/tnavarrete-iedib/bigdata-24-25/raw/refs/heads/main/reviews.csv.gz)

#### 3.4. Tell which are the 10 users (_reviewer\_id_) who have written the most comments on the accommodations that have a license (_license_ column), sorted from most to least comments.

#### 3.5. Calculate how many comments there are for each municipality on the island (you cannot use the _number\_of\_reviews_ column in the listings file, you have to calculate it from the data in the _reviews_ file). Sort them in descending order by the number of comments.